# 28.07.2023

# Барков Михаил

# Интеграция DGL в JanusGraph

## Что сделано за эту неделю

Были внесены изменения в сервис обработки данных. Оптимизирован парсер данных для категориальных признаков, добавлено сохранение отображения меток категорий в изначальное название категории, добавлены новые типы кодирования признаков, часть функциональности перенесена из S3Service в ProcessingService. В сервис обучения были внесены изменения, следующие из изменений в сервисе обработки данных.  
Был частично реализован endpoint-сервиса. В ходе разработки возникли вопросы. Для продолжения разработки было бы полезно получить рекомендации по выбору путей решения возникнувших проблем.

## Вопросы

#### 1. Хранение моделей

Я вижу несколько способов хранить обученные модели, используемые endpoint-сервисом.

1. Можно загружать их в локальное хранилище на сервер с endpoint-сервисом.
2. Можно хранить их в S3.  

Раз уж в проекте уже используется объектное хранилище, то разумно было бы статические данные хранить там, а не придумывать специфическое решение. Поэтому встаёт вопрос, как хранить данные обученной модели. 

1. Можно хранить python-объект и загружать его в endpoint-сервис. Загружать и исполнять код из стороннего источника небезопасно, но эта проблема должна решится, если хранить модели в приватном бакете, к которому нет доступа у пользователя (т.е. если мы уверены, что объект создан нашим сервисом и не был изменён пользователем). 
2. Можно хранить веса обученной модели в объектном хранилище. Тогда пользователь может получить веса модели, что может быть полезно, если пользователь захочет использовать приложение для обучения модели, но ему не нужно использовать endpoint-сервис. Чтобы сделать это с первым вариантом нужно будет создавать дополнительный функционал (например, позволять пользователю выгружать веса модели по запросу к сервису обучения).

Хранение весов выглядит намного более привлекательным, чем хранение готовых объектов. Нет лишнего расхода памяти на хранение ненужной информации, более безопасно, не требует реализации обновления хранимых объектов при внесении изменений в классы их моделей. Но у такого решения есть большая проблема. Весами должна инициализироваться модель, а класс модели, предоставляющий интерфейс для прогнозирования, находится в сервисе обучения. Я вижу несколько решений.

1. Создание небольшой библиотеки, хранящей классы моделей и сопутствующие им (классы типа samplers, dataloaders и т.д.).
2. Хранение классов моделей в объектном хранилище и загрузка этих классов при инициализации сервисов обучения и эндпоинта (это будет работать на перспективу добавления пользовательских моделей).
3. Можно сделать ручку у сервиса обучения для инициализации модели по весам, но это выглядит совсем плохо. 

Я склоняюсь ко второму варианту, но в этом вопросе я больше всего не уверен, и мне кажется, что адекватное решение я не вижу.  
  
Хотелось бы увидеть вашу оценку первых двух пунктов и совет по третьему, т.к. в нём я очень сильно сомневаюсь.

#### 2. Работа с endpoint-сервисом

Запросы на прогнозирование можно обрабатывать двумя способами.
  
1. Использование стратегии обхода. Для реализации такого решения с помощью gremlin-python нужно будет создать класс стратегии и использовать кастомный Traversal класс. В gremlin-python есть проблема с использованием собственных стратегий. Существующие стратегии являются простыми заглушками и используются только кодирования в байт-коде и передачи на gremlin-server для повторного построения на стороне сервера. Поэтому придётся "маскировать" использование стратегии.   
2. Использование инструкции call. Тоже можно реализовать, используя кастомный Traversal класс.   
  
Реализация второго подхода должна быть проще и выглядит правильнее. Кажется, что первый способ предоставляет более удобный функционал, но я могу ошибаться.  
  
Если это возможно, хотел бы попросить ссылку на какой-нибудь пример реализации call сервиса или какой-нибудь проект, где можно увидеть примеры использования или построения подобных сервисов.

#### 3. Запросы к endpoint-сервису

Для реализации хотелось бы знать, в каком виде запросы должны приходить к сервису. Байткод или просто текст gremlin-запроса? 

#### 4. Связь между сервисами

Когда в сервис приходит запрос на прогнозирование свойства сущности, требуется загрузить граф (для задачи трансдуктивного обучения) или подграф (для задачи индуктивного обучения) из базы данных. Делать это я предлагаю с использованием функционала сервисов экспорта и обработки данных.  
Было бы неразумно реализовывать загрузку данных из JanusGraph в endpoint-сервисе. Во-первых, это было бы дублированием функционала остальных сервисов. Во-вторых, это не должно быть функционалом этого сервиса.  
Получается, сервис экспорта данных необходимо дополнить функционалом загрузки подграфов, работающий аналогично используемому DataLoader-классу из DGL.   

## Организационные вопросы

#### 2.1. Форма отчётности

Скажите, пожалуйста, удобен ли такой формат отчётов или лучше предоставлять отчёты как-то иначе? 

#### 2.2. Тестирование

Удобно ли будет, если я создам докер-образ для разворачивания сервисов и FakeS3, JanusGraphDB с загруженными данными датасета из примера будет развернута у меня на сервере, и добавлю юпитер-блокнот для проведения экспорта, обработки и обучения данных пошагово?   

#### 2.3. Описание реализации

Ещё хотелось бы узнать, нужно ли предоставить детальное описание внутреннего устройства уже реализованных сервисов. Если бы я предоставил такое описание, то вы могли бы указать на мои ошибки. Раньше я не предоставлял такое описание, т.к. код часто переписывался и текст быстро потерял бы актуальность, но сейчас (вроде как) решения устоялись и уже представляют из себя что-то осмысленное. 

## Что дальше

В ближайшее время я планирую, получив ваши рекомендации, реализовать endpoint-сервис. После этого можно будет подумать о расширении доступных методов обучения. Но, судя по прошлому опыту планирования, заглядывать далеко не стоит. 